In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Human Identification Review

/content/drive/MyDrive/Human Identification Review


In [3]:
!ls

 best.pt
 best_zoomed_image.png
 boats.jpg
'dental data( Original Raw)'
 dental_tesing.ipynb
 diagram.drawio
'DoCM Checker'
'DoCM( Degree of Change Measurement) dataset'
'DoCM_ Result_with_tp_fpfor10image.csv'
'edited(IdC)'
 HID.ipynb
 HID-updated.ipynb
'Human_ID(Benchmarking).ipynb'
'IdC- (Identification Checker Dataset)'
'IdC_ Result_with_tp_fp.csv'
'Pipeline Images'
 Reference
 result.csv
 result_for_thresholding.csv
 result_for_thresholding_shift.csv
 result_for_zoom_corrected.csv
 result_for_zoom_corrected_shift.csv
 result-previous.csv
 result_shift.csv
 Scaling-SSIM.ipynb
'updated_HID(raw).ipynb'
'Zoom corrected'
'Zoom in'


In [4]:
!pip install ultralytics -q
!pip install shapely -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.2 MB/s eta 0:00:00


In [5]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
from scipy.spatial import distance
from skimage.metrics import structural_similarity as ssim
model = YOLO('best.pt')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [6]:
def ssim_scale(gt_img, input_img, save=False, show=False):
    # Resize input to match GT dimensions if needed (optional)
    gt_img = cv2.cvtColor(gt_img, cv2.COLOR_BGR2GRAY)
    input_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
    input_img = cv2.resize(input_img, (gt_img.shape[1], gt_img.shape[0]))

    def center_zoom(img, scale):
        h, w = img.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, 0, scale)
        zoomed = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_LINEAR)
        return zoomed

    # Search for the best scale using SSIM
    best_score = -1
    best_scale = 1.0
    best_zoomed = None

    for scale in np.linspace(0.8, 1.2, 100):
        zoomed = center_zoom(input_img, scale)
        score, _ = ssim(gt_img, zoomed, full=True)

        if score > best_score:
            best_score = score
            best_scale = scale
            best_zoomed = zoomed

    # Generate overlay and difference image
    overlay = cv2.addWeighted(gt_img, 0.5, best_zoomed, 0.5, 0)
    diff = cv2.absdiff(gt_img, best_zoomed)

    if save:
        # Save results
        cv2.imwrite('best_zoomed_image.png', best_zoomed)
        cv2.imwrite('overlay_image.png', overlay)
        cv2.imwrite('difference_image.png', diff)

    if show:
        # Display results
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 3, 1)
        plt.title(f'Best Zoomed (scale={best_scale:.3f})')
        plt.imshow(best_zoomed, cmap='gray')

        plt.subplot(1, 3, 2)
        plt.title('Overlay')
        plt.imshow(overlay, cmap='gray')

        plt.subplot(1, 3, 3)
        plt.title('Difference')
        plt.imshow(diff, cmap='hot')

        plt.tight_layout()
        plt.show()

    cv2.imwrite('best_zoomed_image.png', best_zoomed if best_zoomed is not None else input_img)
    best_zoomed = cv2.imread('best_zoomed_image.png')
    return best_zoomed

def sol1(i1, i2, do_ssim=True, do_shift=True):
    if do_ssim:
        i2 = ssim_scale(i1, i2, save=False, show=False)

    # Perform inference on both images
    results1 = model(i1)
    results2 = model(i2)

    # Extract OBB data from predictions
    obb1 = results1[0].obb
    obb2 = results2[0].obb

    # Convert boxes (xywhr) and classes to NumPy arrays
    boxes1 = obb1.xywhr.cpu().numpy()  # [x_center, y_center, width, height, rotation]
    classes1 = obb1.cls.cpu().numpy()  # Class IDs (0 to 15)
    boxes2 = obb2.xywhr.cpu().numpy()
    classes2 = obb2.cls.cpu().numpy()

    # Initialize dictionaries for grouping boxes by class
    boxesup1_by_class = defaultdict(list)
    boxesup2_by_class = defaultdict(list)
    boxesdown1_by_class = defaultdict(list)
    boxesdown2_by_class = defaultdict(list)

    # Group boxes by class for image 1
    for i, c in enumerate(classes1):
        if int(c) % 2 == 0:
            boxesdown1_by_class[int(c)].append(boxes1[i])
        else:
            boxesup1_by_class[int(c)].append(boxes1[i])

    # Group boxes by class for image 2
    for i, c in enumerate(classes2):
        if int(c) % 2 == 0:
            boxesdown2_by_class[int(c)].append(boxes2[i])
        else:
            boxesup2_by_class[int(c)].append(boxes2[i])

    # Function to compute corner points from xywhr format
    def get_corners(xywhr):
        x, y, w, h, r = xywhr
        corners = [
            [x - w / 2, y - h / 2],
            [x + w / 2, y - h / 2],
            [x + w / 2, y + h / 2],
            [x - w / 2, y + h / 2]
        ]
        cos_r = np.cos(r)
        sin_r = np.sin(r)
        rotated_corners = []
        for px, py in corners:
            px -= x
            py -= y
            px_rot = px * cos_r - py * sin_r
            py_rot = px * sin_r + py * cos_r
            rotated_corners.append([px_rot + x, py_rot + y])
        return np.array(rotated_corners)

    # Create pairs for "up" group (odd classes) using nearest-neighbor matching
    pairs_up = []
    for c in range(1, 16, 2):
        if boxesup1_by_class[c] and boxesup2_by_class[c]:
            centroids1 = np.array([box[:2] for box in boxesup1_by_class[c]])
            centroids2 = np.array([box[:2] for box in boxesup2_by_class[c]])
            dist_matrix = distance.cdist(centroids2, centroids1)
            for i in range(len(centroids2)):
                min_idx = np.argmin(dist_matrix[i])
                if dist_matrix[i, min_idx] < 50:  # Distance threshold
                    box2 = boxesup2_by_class[c][i]
                    box1 = boxesup1_by_class[c][min_idx]
                    idx1 = np.where((boxes1 == box1).all(axis=1))[0][0]
                    idx2 = np.where((boxes2 == box2).all(axis=1))[0][0]
                    pairs_up.append((box1, box2, idx1, idx2))

    # Create pairs for "down" group (even classes) using nearest-neighbor matching
    pairs_down = []
    for c in range(0, 16, 2):
        if boxesdown1_by_class[c] and boxesdown2_by_class[c]:
            centroids1 = np.array([box[:2] for box in boxesdown1_by_class[c]])
            centroids2 = np.array([box[:2] for box in boxesdown2_by_class[c]])
            dist_matrix = distance.cdist(centroids2, centroids1)
            for i in range(len(centroids2)):
                min_idx = np.argmin(dist_matrix[i])
                if dist_matrix[i, min_idx] < 50:  # Distance threshold
                    box2 = boxesdown2_by_class[c][i]
                    box1 = boxesdown1_by_class[c][min_idx]
                    idx1 = np.where((boxes1 == box1).all(axis=1))[0][0]
                    idx2 = np.where((boxes2 == box2).all(axis=1))[0][0]
                    pairs_down.append((box1, box2, idx1, idx2))

    # Function to calculate average shift between centroids
    def calculate_avg_shift(pairs):
        if not pairs:
            return np.array([0, 0], dtype=np.float32)
        centroids1 = np.array([box1[:2] for box1, _, _, _ in pairs], dtype=np.float32)
        centroids2 = np.array([box2[:2] for _, box2, _, _ in pairs], dtype=np.float32)
        return np.mean(centroids1 - centroids2, axis=0)

    # Calculate average shifts
    avg_shift_up = calculate_avg_shift(pairs_up)
    avg_shift_down = calculate_avg_shift(pairs_down)

    # Function to shift a box
    def shift_box(box, shift):
        shifted_box = box.copy()
        shifted_box[:2] += shift
        return shifted_box

    # Collect all boxes for each group
    boxesup1 = [box for box, cls in zip(boxes1, classes1) if int(cls) % 2 != 0]  # Odd classes
    boxesup2 = [box for box, cls in zip(boxes2, classes2) if int(cls) % 2 != 0]
    boxesdown1 = [box for box, cls in zip(boxes1, classes1) if int(cls) % 2 == 0]  # Even classes
    boxesdown2 = [box for box, cls in zip(boxes2, classes2) if int(cls) % 2 == 0]

    # Compute shifted boxes for all boxes in image 2
    shifted_boxesup2 = [shift_box(box, avg_shift_up) for box in boxesup2]
    shifted_boxesdown2 = [shift_box(box, avg_shift_down) for box in boxesdown2]

    # Get corners for all boxes
    corners_box1_up = [get_corners(box) for box in boxesup1]
    corners_box2_up = [get_corners(box) for box in boxesup2]
    corners_shifted_box2_up = [get_corners(box) for box in shifted_boxesup2]
    corners_box1_down = [get_corners(box) for box in boxesdown1]
    corners_box2_down = [get_corners(box) for box in boxesdown2]
    corners_shifted_box2_down = [get_corners(box) for box in shifted_boxesdown2]

    # Function to calculate IoUs for paired boxes
    def calculate_ious(pairs, avg_shift, classes1, corners1_list, corners2_list, corners2_shifted_list):
        ious = []
        for idx, (box1, box2, idx1, idx2) in enumerate(pairs):
            corners1 = get_corners(box1)
            corners1_list.append(corners1)
            corners2 = get_corners(box2)
            corners2_list.append(corners2)
            shifted_box2 = shift_box(box2, avg_shift)
            corners2_shifted = get_corners(shifted_box2)
            corners2_shifted_list.append(corners2_shifted)
            poly1 = Polygon(corners1)
            poly2 = Polygon(corners2_shifted) if do_shift else Polygon(corners2)
            intersection = poly1.intersection(poly2).area
            union = poly1.union(poly2).area
            iou = intersection / union if union > 0 else 0
            ious.append(iou)
        return ious

    # Lists for IoU calculation (only for paired boxes)
    corners_for_box1_up = []
    corners_for_box2_up = []
    corners_for_box2_shifted_up = []
    corners_for_box1_down = []
    corners_for_box2_down = []
    corners_for_box2_shifted_down = []

    # Calculate IoUs
    ious_up = calculate_ious(pairs_up, avg_shift_up, classes1, corners_for_box1_up, corners_for_box2_up, corners_for_box2_shifted_up)
    ious_down = calculate_ious(pairs_down, avg_shift_down, classes1, corners_for_box1_down, corners_for_box2_down, corners_for_box2_shifted_down)

    # Compute mean IoUs
    mean_iou_up = np.mean(ious_up) if ious_up else 0
    mean_iou_down = np.mean(ious_down) if ious_down else 0
    overall_ious = ious_up + ious_down
    mean_iou_overall = np.mean(overall_ious) if overall_ious else 0


    # # Visualization with all boxes
    # fig, axes = plt.subplots(2, 2, figsize=(18, 18))

    # # Original Up Boxes
    # axes[0, 0].set_title("Original Up Boxes (Odd Classes)")
    # for box in corners_box1_up:
    #     box = np.vstack([box, box[0]])
    #     axes[0, 0].plot(box[:, 0], box[:, 1], 'b-', lw=2, label='Image 1' if box is corners_box1_up[0] else "")
    # for box in corners_box2_up:
    #     box = np.vstack([box, box[0]])
    #     axes[0, 0].plot(box[:, 0], box[:, 1], 'r-', lw=2, label='Image 2' if box is corners_box2_up[0] else "")

    # # Shifted Up Boxes
    # axes[0, 1].set_title("Shifted Up Boxes (Odd Classes)")
    # for box in corners_box1_up:
    #     box = np.vstack([box, box[0]])
    #     axes[0, 1].plot(box[:, 0], box[:, 1], 'b-', lw=2, label='Image 1' if box is corners_box1_up[0] else "")
    # for box in corners_shifted_box2_up:
    #     box = np.vstack([box, box[0]])
    #     axes[0, 1].plot(box[:, 0], box[:, 1], 'r-', lw=2, label='Image 2 Shifted' if box is corners_shifted_box2_up[0] else "")

    # # Original Down Boxes
    # axes[1, 0].set_title("Original Down Boxes (Even Classes)")
    # for box in corners_box1_down:
    #     box = np.vstack([box, box[0]])
    #     axes[1, 0].plot(box[:, 0], box[:, 1], 'b-', lw=2, label='Image 1' if box is corners_box1_down[0] else "")
    # for box in corners_box2_down:
    #     box = np.vstack([box, box[0]])
    #     axes[1, 0].plot(box[:, 0], box[:, 1], 'r-', lw=2, label='Image 2' if box is corners_box2_down[0] else "")

    # # Shifted Down Boxes
    # axes[1, 1].set_title("Shifted Down Boxes (Even Classes)")
    # for box in corners_box1_down:
    #     box = np.vstack([box, box[0]])
    #     axes[1, 1].plot(box[:, 0], box[:, 1], 'b-', lw=2, label='Image 1' if box is corners_box1_down[0] else "")
    # for box in corners_shifted_box2_down:
    #     box = np.vstack([box, box[0]])
    #     axes[1, 1].plot(box[:, 0], box[:, 1], 'r-', lw=2, label='Image 2 Shifted' if box is corners_shifted_box2_down[0] else "")

    # # Formatting
    # for ax in axes.flatten():
    #     ax.set_xlabel("X")
    #     ax.set_ylabel("Y")
    #     ax.set_aspect('equal', adjustable='box')
    #     ax.invert_yaxis()
    #     ax.grid(True)
    #     if ax == axes[0, 0] or ax == axes[0, 1] or ax == axes[1, 0] or ax == axes[1, 1]:
    #         ax.legend()

    # plt.tight_layout()
    # plt.show()

    return mean_iou_overall

In [7]:
import pandas as pd
import cv2
import glob
import os

# Create an empty list to store rows
rows = []
do_ssim = False     # enable SSIM-based zoom
do_shift = False     # jaw-wise shift

for i in range(50, 101, 5):  # 50 to 100, step 5
    threshold = i / 100
    match_count = 0
    matched_number = []
    checker_path = "/content/drive/MyDrive/Human Identification Review/IdC- (Identification Checker Dataset)"
    docm_path = "/content/drive/MyDrive/Human Identification Review/edited(IdC)"
    # checker_path = "/content/drive/MyDrive/Human Identification Review/DoCM Checker"
    # docm_path = "/content/drive/MyDrive/Human Identification Review/DoCM( Degree of Change Measurement) dataset"

    # Get all images from both folders
    checker_images = sorted(glob.glob(os.path.join(checker_path, "*.jpg")))
    docm_images = sorted(glob.glob(os.path.join(docm_path, "*.jpg")))
    # Loop through each checker image
    for img1_path in checker_images:
        print(f"Processing Checker image: {img1_path}")
        img1 = cv2.imread(img1_path)

        for img2_path in docm_images:
            img2 = cv2.imread(img2_path)
            x = sol1(img1, img2, do_ssim=do_ssim, do_shift=do_shift)
            if x >= threshold:
                match_count += 1
                matched_number.append([os.path.splitext(os.path.basename(img1_path))[0],
                os.path.splitext(os.path.basename(img2_path))[0]])

    rows.append([threshold, match_count, matched_number])

# Convert to DataFrame
df = pd.DataFrame(rows, columns=['threshold', 'match count', 'matched number'])

# Save to CSV
df.to_csv(f'result_for_IdC{"_ssim" if do_ssim else ""}{"_shift" if do_shift else ""}.csv', index=False)

Streaming output truncated to the last 5000 lines.
Speed: 2.6ms preprocess, 10.0ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 None10.2ms
Speed: 1.9ms preprocess, 10.2ms inference, 3.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 None10.0ms
Speed: 2.2ms preprocess, 10.0ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 None10.1ms
Speed: 2.0ms preprocess, 10.1ms inference, 3.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 None11.1ms
Speed: 2.5ms preprocess, 11.1ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 None10.9ms
Speed: 1.9ms preprocess, 10.9ms inference, 3.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 None10.2ms
Speed: 2.2ms preprocess, 10.2ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 None10.4ms
Speed: 1.9ms preprocess, 10.4ms inference, 3.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 

In [ ]:
import numpy as np
import pandas as pd
import ast
from collections import Counter

file_path = '/content/result_for_IdC.csv'
df = pd.read_csv(file_path)

thresholds = sorted([round(float(_), 2) for _ in df['threshold']])


all_pairs = []
for matches in df['matched number']:
    pairs = ast.literal_eval(matches)
    all_pairs.extend([p[0] for p in pairs])

unique_gt = sorted(set(all_pairs))


result_df = pd.DataFrame(index=unique_gt, columns=thresholds)


for _, row in df.iterrows():
    threshold = round(row['threshold'], 2)
    pairs = ast.literal_eval(row['matched number'])
    for real, pred in pairs:
        current_val = result_df.loc[real, threshold]
        if pd.isna(current_val) or current_val == 0:
            result_df.loc[real, threshold] = str(pred)
        else:
            result_df.loc[real, threshold] = str(current_val) + f", {pred}"

result_df = result_df.fillna(0)


TP_counts = {t: 0 for t in thresholds}
FP_counts = {t: 0 for t in thresholds}
FN_counts = {t: 0 for t in thresholds}

for col in thresholds:
    matched_gts = set()
    for gt in unique_gt:
        val = result_df.loc[gt, col]
        if val != 0:
            preds = [v.strip() for v in str(val).split(",") if v.strip() != "0"]
            for p in preds:
                pred_id = p.split('e')[0]  # Take part before 'e'
                if pred_id == str(gt):
                    TP_counts[col] += 1
                    matched_gts.add(gt)
                else:
                    FP_counts[col] += 1
    FN_counts[col] = len(unique_gt) - len(matched_gts)

# Append TP, FP, FN as new rows
result_df.loc["TP"] = [TP_counts[t] for t in thresholds]
result_df.loc["FP"] = [FP_counts[t] for t in thresholds]
result_df.loc["FN"] = [FN_counts[t] for t in thresholds]

TP = 0
FP = 0
TP_list = []
FP_list = []

for gt, row in result_df.iterrows():
    matches = []
    for col in thresholds:
        val = row[col]
        if val != 0:
            if isinstance(val, str) and "," in val:
                matches.extend(val.split(","))
            else:
                matches.append(str(val))
    matches = [m.strip() for m in matches if m != '0']

    if any(m.startswith(str(gt)) for m in matches):
        TP += 1
    else:
        FP += 1

    TP_list.append(TP)
    FP_list.append(FP)

result_df["TP"] = TP_list
result_df["FP"] = FP_list


def clean_predictions(val):
    if val == 0 or pd.isna(val):
        return 0
    preds = [v.strip().split('e')[0] for v in str(val).split(',')]
    preds = sorted(set(preds))
    return ", ".join(preds)

for col in thresholds:
    result_df[col] = result_df[col].apply(clean_predictions)


result_df.to_csv("Final_Output_Clean.csv", index=True)
result_df.drop(1.0, axis=1, inplace=True)
result_df
result_df.to_csv('Final_Output clean.csv')


In [ ]:
import matplotlib.pyplot as plt

# Data from your table
iou = [0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95]
precision = [0.127, 0.263, 0.521, 0.754, 0.98, 1.0, 1.0, 1.0, 1.0, 1.0]
recall = [0.98, 0.98, 0.98, 0.98, 0.98, 0.98, 0.98, 0.98, 0.96, 0.68]

# ----- ROC Curve -----
fpr_proxy = [1-p for p in precision]  # using 1-precision as proxy for FPR

plt.figure(figsize=(6,6))
plt.plot(fpr_proxy, recall, marker='o', label="ROC Curve")
plt.plot([0,1],[0,1],'--',color='gray',label="Random Guess")
plt.xlabel("False Positive Rate (1 - Precision)")
plt.ylabel("True Positive Rate (Recall)")
plt.title("ROC Curve (Approx using Precision)")
plt.legend()
plt.grid(True)
plt.show()

# ----- CMC Curve -----
plt.figure(figsize=(6,6))
plt.plot(iou, recall, marker='o')
plt.xlabel("IoU Threshold")
plt.ylabel("Cumulative Match (Recall)")
plt.title("CMC Curve")
plt.grid(True)
plt.show()


# Plot Precision & Recall vs IoU on the same graph
plt.figure(figsize=(8,6))

plt.plot(iou, precision, marker='o', linestyle='-', color='b', label="Precision")
plt.plot(iou, recall, marker='s', linestyle='--', color='g', label="Recall")

plt.xlabel("IoU Threshold")
plt.ylabel("Score")
plt.title("Precision and Recall vs IoU")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import ast
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt

file_path = 'F:/Human Identification Review/result_for_thresholding.csv'
df = pd.read_csv(file_path)

thresholds = sorted([round(float(_), 2) for _ in df['threshold']])

# Collect all unique ground truth IDs
all_pairs = []
for matches in df['matched number']:
    pairs = ast.literal_eval(matches)
    all_pairs.extend([p[0] for p in pairs])

unique_gt = sorted(set(all_pairs))

# Initialize result dataframe
result_df = pd.DataFrame(index=unique_gt, columns=thresholds)

# Fill in the predicted values
for _, row in df.iterrows():
    threshold = round(row['threshold'], 2)
    pairs = ast.literal_eval(row['matched number'])
    for real, pred in pairs:
        current_val = result_df.loc[real, threshold]
        if pd.isna(current_val) or current_val == 0:
            result_df.loc[real, threshold] = str(pred)
        else:
            result_df.loc[real, threshold] = str(current_val) + f", {pred}"

result_df = result_df.fillna(0)

# Replace threshold columns with most frequent value(s) in each row
for gt, row in result_df.iterrows():
    # Collect all values > 0
    values = []
    for col in thresholds:
        val = row[col]
        if val != 0:
            if isinstance(val, str) and "," in val:
                values.extend(val.split(","))
            else:
                values.append(str(val))
    values = [v.strip() for v in values if v != '0']

    if values:
        # Count frequencies
        freq_counter = Counter(values)
        max_freq = max(freq_counter.values())
        most_freq_values = [v for v, count in freq_counter.items() if count == max_freq]
        # Join them if multiple
        replacement = ", ".join(sorted(most_freq_values))

        # Replace each threshold column that is >0
        for col in thresholds:
            if row[col] != 0:
                result_df.loc[gt, col] = replacement

# Now calculate TP and FP as before
TP = 0
FP = 0
TP_list = []
FP_list = []

for gt, row in result_df.iterrows():
    matches = []
    for col in thresholds:
        val = row[col]
        if val != 0:
            if isinstance(val, str) and "," in val:
                matches.extend(val.split(","))
            else:
                matches.append(str(val))
    matches = [m.strip() for m in matches if m != '0']

    if str(gt) in matches:
        TP += 1
    else:
        FP += 1

    TP_list.append(TP)
    FP_list.append(FP)

# result_df["TP"] = TP_list
# result_df["FP"] = FP_list

result_df
#result_df.to_csv('Final_Output_filtered_thresholding.csv')
data = {"Image": list(result_df.index)}

# Loop through thresholds
for col in result_df.columns:
    data[col] = [str(val) if val != 0 else "0" for val in result_df[col]]

# Optional: convert back into DataFrame for easier viewing
new_df = pd.DataFrame(data).set_index("Image")

df_binary = new_df.applymap(lambda x: 1 if x != "0" else 0)

# --- Heatmap ---
plt.figure(figsize=(15,9))

# Increase font scale
sns.set(font_scale=1.5)  # Adjust scale (1.5 → bigger text)

ax = sns.heatmap(
    df_binary, annot=True, fmt="d", cmap="Greens",
    cbar_kws={"label": "Detection"}, annot_kws={"size":24}
)

# Increase title and labels
plt.title("Detection Across IoU Threshold", fontsize=24)
plt.xlabel("IoU Threshold", fontsize=24)
plt.ylabel("Image Number", fontsize=24)

# Increase tick size
ax.tick_params(axis="both", labelsize=24)

# Increase colorbar font size
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=24)
cbar.set_label("Detection", fontsize=24)

plt.show()